In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import traceback
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Filter subtitle

In [309]:
subtitles_to_remove = ['VR',
                       "WWE",
                       "Инсульт",
                       "Погладь",
                       "Fishing",
                       "Football",
                       "NBA",
                       "Golf",
                       "Let's Sing",
                       "MLB",
                       "My Universe",
                       "ACA NEOGEO",
                       "LEGO",
                       "Outbreak",
                       "The Escapists",
                       "Madden",
                       "NHL",
                       "Chess",
                       "Arcade Archives",
                       "FINAL FANTASY",
                       "Syberia",
                       "Assassin's Creed",
                       "Tennis",
                       "MFC",
                       "Disney",
                       "Pool",
                       "Tycoon",
                       "Погладить",
                       "STAR WARS",
                       "Call of Duty",
                       "Snooker",
                       "Tom Clancy's",
                       "Star Wars",
                       "Soccer",
                       "Baseball",
                       "Mega Man",
                       "NARUTO",
                       "Borderlands",
                       "Cricket",
                       "Саундтрек",
                       "Warhammer",
                       "South Park",
                       "Solitaire",
                       "EA SPORTS",
                       "Skyrim",
                       "Skate",
                       "JIGSAW",
                       "Poker",
                       "The Walking Dead",
                       "Dungeon",
                       
                      ]

In [310]:
title_substring = subtitles_to_remove[-1]

filename = "full_list_v2.csv"
df = pd.read_csv(filename)

for subtitle in subtitles_to_remove:
    df.loc[df[df['title'].str.contains(title_substring)].index, 'status'] = -1

df = df.sort_values(by=['status', 'title'], ascending=[False, True]).reset_index(drop=True)
df.to_csv(filename, index=False)


In [311]:
title_substring = subtitles_to_remove[-1]
df[df['title'].str.contains(title_substring)]

,title,platform,psd_URL,icon_URL,price,status
1260,Ancient Dungeon VR,PS5,https://psdeals.net/ua-store/game/2693826/anci...,https://store.playstation.com/store/api/chihir...,649.00₴,-1
2215,Bit Dungeon Plus,PS5,https://psdeals.net/ua-store/game/2761567/bit-...,https://store.playstation.com/store/api/chihir...,299.00₴,-1
2216,Bit Dungeon Plus [Cross-Buy],PS4 / PS Vita,https://psdeals.net/ua-store/game/1537054/bit-...,https://store.playstation.com/store/api/chihir...,229.00₴,-1
2436,Boyfriend Dungeon,PS5,https://psdeals.net/ua-store/game/2543582/boyf...,https://store.playstation.com/store/api/chihir...,649.00₴,-1
2940,Chocobo's Mystery Dungeon EVERY BUDDY!,PS4,https://psdeals.net/ua-store/game/1756563/choc...,https://store.playstation.com/store/api/chihir...,"1,399.00₴",-1
...,...,...,...,...,...,...
11019,The Dungeon of Naheulbeuk: The Amulet of Chaos...,PS5 / PS4,https://psdeals.net/ua-store/game/2310481/the-...,https://store.playstation.com/store/api/chihir...,649.00₴,-1
11987,Trickster VR: Co-op Dungeon Crawler,PS4,https://psdeals.net/ua-store/game/1655080/tric...,https://store.playstation.com/store/api/chihir...,519.00₴,-1
11992,Trigger Dungeon,PS5,https://psdeals.net/ua-store/game/2821464/trig...,https://store.playstation.com/store/api/chihir...,129.00₴,-1
11993,Trigger Dungeon,PS4,https://psdeals.net/ua-store/game/2626959/trig...,https://store.playstation.com/store/api/chihir...,129.00₴,-1


# Delete publisher

In [5]:
def get_pub(publisher):
    
    time = dt.now()
    
    page_num = 1
    df_pub = pd.DataFrame(columns=['title', 'platform', "psd_URL", "icon_URL", "price"])

    prev_page = None

    while True:
        try:
            URL = f"https://psdeals.net/ua-store/all-games/{page_num}?publisher={publisher}"
            #URL = f"https://psdeals.net/ua-store/all-games{page_num}?contentType%5B%5D=games&contentType%5B%5D=bundles&feature%5B%5D=vrGames"

            headers = requests.utils.default_headers()
            page = requests.get(URL, headers=headers)

            soup = BeautifulSoup(page.content, "html.parser")

            grid_items = soup.find_all('div', class_='game-collection-item col-md-2 col-sm-4 col-xs-6')

            if grid_items == prev_page:
                break

            for item in grid_items:

                title = item.find('span', class_='game-collection-item-details-title').text
                platform = item.find('span', class_='game-collection-item-top-platform').text
                image = item.find('img', class_='game-collection-item-image lazy').get('data-src')
                psd_link = "https://psdeals.net" + item.find('a', class_='game-collection-item-link').get('href')

                try:
                    price = item.find('span', class_='game-collection-item-price').text
                except:
                    price = np.NaN

                new_row = {"title": title, 
                           "platform": platform,
                           "price": price,
                           "psd_URL": psd_link,
                           "icon_URL": image}


                df_pub = df_pub.append(new_row, ignore_index=True)

            print(f"page {page_num}  time {dt.now() - time}")

            page_num += 1

            prev_page = grid_items

        except Exception as e:
            print(e)
            print(traceback.format_exc())
            break
            
    return df_pub

In [332]:
bad_publishers_list = ["Zakym+s.+r.+o.",
                       "ThiGames", 
                       "Kimulator%27s+Films+inc.", 
                       "SILESIA+GAMES+SP.+Z+O.O.",
                       "eastasiasoft",
                       "e-llusiontertainment",
                       "Webnetic+s.+r.+o.",
                       "Klovako+s.+r.+o.",
                       "THE+QUANTUM+ASTROPHYSICISTS+GUILD",
                       "SMOBILE+INC",
                       "Gametry+sp.+z+o.+o.",
                       "NOSTRA+GAMES+LTD",
                       "Benoit+Varasse+trading+as+Pix+Arts",
                       "Wild+River+Games",
                       "Acyntha",
                       "SOURCE+BYTE+SPÓŁKA+Z+OGRANICZONĄ+ODPOWIEDZIALNOŚCIĄ",
                       "Toplitz+Productions",
                       "Breakthrough+Gaming+LLC",
                       "YASH+FUTURE+TECH+SOLUTIONS+PVT+LTD",
                       "JOINDOTS+GMBH",
                       "4HIT",
                       "OCEAN+MEDIA+LLC",
                       "QUBYTE+INTERACTIVE",
                       "QUBICGAMES+SP.+Z+O.O.",
                       "Legacy+Interactive%2C+Inc.",
                       "Interactive+Dreams+Limited",
                       "ERIK+GAMES+YAZILIM+OYUN+TEKNOLOJILERI+TICARET+ANONIM+SIRKETI",
                       "Gameloft+SE",
                       "COATSINK+SOFTWARE+LTD",
                       "Thunderful+Games+Ltd",
                       "THE+VOICES+GAMES+LTD",
                       "PLAYSTIGE+INTERACTIVE+INFORMATION+AND+TRADE+LIMITED+LIABILITY+COMPANY",
                       "Aristo+Studio+s.r.o.",
                       "4HIT+LTD",
                       "KAIROSOFT+CO.%2CLTD",
                       "CFK+Co.%2C+Ltd.",
                       "ERDEM+SEN",
                       "KLABATER",
                       "ARCADE+DISTILLERY",
                       "OUTRIGHT+GAMES+LLC",
                       "KODOBUR+YAZILIM+BILISIM+TASARIM+ARG",
                       "Volkov+Konstantin+Sergeevich",
                       "TESEO+Y+EL+MINOTAURO%2C+S.L.U.",
                       "WEAKFISH+STUDIO+PUBLISHING",
                       "EVGENIY+KOLPAKOV",
                       "Gamuzumi",
                       "FOREVER+ENTERTAINMENT+S.+A.",
                       "Bigben+Interactive",
                       "MARKT%2BTECHNIK+VERLAG+GMBH",
                       "Scout+Game+Studios+Sociedad+Anonima",
                       "Xitilon",
                       "Baltoro+Games+Sp.z+o.o.",
                       "Armin+Unold",
                       "MOVING+PLAYER+SAS",
                       "Mike+Toth+trading+as+Toth+Games",
                       "FUNBOX+MEDIA+LIMITED",
                       "ARMOR+GAMES+INC",
                       "DIGERATI+DISTRIBUTION",
                       "CHUBBY+PIXEL",
                       "S.R.L.+%22RANDOMSPIN-GAMES%22",
                       "ARTIFEX+MUNDI+S.A.",
                       "Xeneder+Team",
                       "Revulo+Games+sp.+z+o.+o.",
                       "Skermunkel+Studios+%28Pty.%29+Ltd.",
                       "KEMCO",
                       "Maksym+Vysochanskyy+trading+as+IndieGames3000",
                       "Lightwood+Games",
                       "Colossus+Game+Studio+LTDA.",
                       "CATNESS+GAME+STUDIOS+SL",
                       "ARTIFEX+MUNDI+SP.+Z+O.O.",
                       "THOMAS+CLAUS+UND+FRANK+MENZEL",
                       "Set+Sail+Games+LLC",
                       "RATALAIKA+GAMES+SL",
                       "2Awesome+Studio++SL",
                       "PARADOX+INTERACTIVE+AB",
                       "Nakana+sp.+z+o.o.",
                       "GAMEMILL+ENTERTAINMENT",
                       "MARMALADE+GAME+STUDIO+LIMITED",
                       "Microids",
                       "BIG+WAY+GAMES",
                       "Dolores+Entertainment+S.L.",
                       "SABEC+LIMITED",
                       "Crunching+Koalas+Sp.+z+o.o.+Sp.+K.",
                       "Phoenix+Reborn+Games",
                       "Samustai+LTD",
                       "UNFINISHED+PIXEL%2C+SL",
                       "Aksys+Games+Localization+Inc.",
                       "Draknek+Limited",
                       "JANDUSOFT+S.L.",
                       "SUPER+POWERUP+GAMES%2C+SOCIEDAD+LIMITADA",
                       "Kanuni+games+oyun+YAZILIM+VE+Pazarlama+Ticaret+limited+Sirketi",
                       "AKUPARA+GAMES",
                       "Whitethorn+Games%2C+Inc.",
                       "ASSEMBLE+ENTERTAINMENT+GMBH",
                       "OÜ+Enningture+Game+Temple",
                       "Oiven+Games",
                       "Rainy+Frog+Co.+Ltd",
                       "Wales+Interactive+Limited",
                       "CHORUS+WORLDWIDE+GAMES",
                       "QUBICGAMES+S.A.",
                       "Curve+Digital",
                       "InfiniteZone+OÜ",
                       "TERNOX",
                       "UKIYO+PUBLISHING+LIMITED",
                       "Limited+Run+Games%2C+Inc.",
                       "EpiXR+Games+UG",
                       "Game+Achievements+Ltd",
                       "FLASHBULB+APS",
                       "FANTASTICO+STUDIO+SRL",
                       "Nellyvision+Ltd",
                       "D3PUBLISHER",
                       "Marvelous+Europe+Limited",
                       "indienova",
                       "VRKiwi",
                       "ULTIMATE+GAMES",
                       "Sometimes+You",
                       "WAYFORWARD+TECHNOLOGIES",
                       "Meridiem+Games+S.L",
                       "Focus+Entertainment&sort=az",
                       "DAEDALIC+ENTERTAINMENT",
                       "ASTRAGON+ENTERTAINMENT+GMBH",
                       "BANDAI+NAMCO+ENTERTAINMENT+EUROPE",
                       "Maximum+Entertainment",
                       "Warlock+Arts",
                       "CHIBIG%2C+S.L.U.",
                       "Afil+Games",
                       "y-zo+studio",
                       "HANDY+GAMES+GMBH",
                       "FusionPlay+GmbH",
                       "GOGAME+CONSOLE+PUBLISHER+LTD",
                       "SANUK+GAMES+SARL",
                       "TINYBUILD+LLC",
                       "MY.GAMES+B.V.",
                       "Puppet+Combo",
                       "CITY-CONNECTION+CO.%2C+LTD.",
                       "Expansive+Worlds+AB",
                       "GameToTop+Corporation+LTD",
                       "PQube+Ltd",
                       "INTRAGAMES+CO.%2C+LTD.",
                       "TOP+HAT+STUDIOS%2C+INC.",
                       "Indie+Remix+Studios+LLC",
                       "Koei+Tecmo+Europe+Ltd",
                       "HEADUP+GAMES+GMBH+CO.+KG",
                       "LOG+GAMES+LTD",
                       "LUCIANO+IURINO+T%2FA+TROGLOBYTES+GAMES",
                       "Progressive+Live+Studio+s.c.",
                       "Motorsport+Games",
                       "Raw+Fury+AB",
                       "ZEROUNO+GAMES",
                       "ESDigital+Games+LTD",
                       "VICTORY+ROAD",
                       "Dead+Drop+Studios+LLC",
                       "Untold+Tales+S.A.",
                       "Slitherine",
                       "PLAYISM",
                       "SAMURAI+PUNK+PTY+LTD",
                       "Zazenfly+Development+DOO",
                       "Ziggurat+Interactive%2C+Inc.",
                       "NUMSKULL+GAMES+LTD",
                       "Feardemic",
                       "indie.io",
                       "Ellada+Games",
                       "SCREENWAVE+MEDIA",
                       "CHETRUSCA+SOFTWORKS+S.R.L",
                       "AEROSOFT+GMBH",
                       "www.handy-games.com+GmbH",
                       "Kistler+Studios",
                       "SOEDESCO+PUBLISHING",
                       "Frozen+Way+S.A.",
                       "RED+ART+GAMES",
                       "KALYPSO+MEDIA",
                       "PLAYWAY+S.A",
                       "Vaka%2C+Inc.",
                       "Reef+Entertainment",
                       "NIS+America",
                       "DANGEN+ENTERTAINMENT",
                       "Disney+Interactive",
                       "ZEN+STUDIOS+LTD",
                       "Green+Man+Gaming+Publishing",
                       "SPIKE+CHUNSOFT%2C+INC.",
                       "DRAGEUS+GAMES+S.A.",
                       "Yogscast+Games+Limited",
                       "RAW+FURY",
                       "FRONTIER+DEVELOPMENTS+PLC",
                       "SURPRISE+ATTACK",
                       "HEADUP+GAMES",
                       "404+GAMES+LTD",
                       "COOL+DEVS+S.R.L",
                       "DANIEL+WENGENROTH",
                       "Gaijin+Distribution+KFT",
                       "ASI+GAMES+TECHNOLOGIES+LTD",
                       "THUNDERFUL+PUBLISHING",
                       "GRIMTALIN+SRL",
                       "Saber+Interactive%2C+Inc",
                       "SKYBOUND+INTERACTIVE+LLC",
                       "LAST+CHANCE+MEDIA",
                       "WIRED+PRODUCTIONS+LIMITED",
                       "SOEDESCO+PUBLISHING+B.V.",
                       "ATARI+EUROPE+S.A.S.U.",
                       "NACON+SA",
                       "FMV+Interactive+Studio+LLC",
                       "JACKBOX+GAMES%2C+INC.",
                       "PLAYTONIC+LTD",
                       "Mob+Entertainment%2C+Inc.",
                       "Midnight+Works+SRL",
                       "BLOWFISH+STUDIOS+PTY+LTD",
                       "UFO+Interactive+Games",
                       "VISION+GAMES+PUBLISHING",
                       "OutOfTheBit+Ltd",
                       "GRAFFITI+GAMES",
                       "VERTIGO+GAMES",
                       "QUBYTE+GAME+STUDIO%2C+LLC",
                       "Frozenbyte+LTD",
                       "MONSTER+COUCH+SP.+Z+O.O.",
                       "ARTAX+GAMES+SL",
                       "Alexander+Carparelli",
                       "AUTO+SLAVIC+d.o.o.",
                       "BeXide",
                       "Juan+Carlos+Moya+Toledo",
                       "Kalypso+Media+GMBH",
                       "Infogrames+LLC",
                       "Iridium+Media+Group+GmbH",
                       "Postmeta+Games+Limited",
                       "XINESS+CO.%2C+LTD",
                       "ONION+GAMES%2CK.K.",
                       "DESERT+OWL+GAMES%2C+LLC",
                       "We+Dig+Games+sp.+z+o.o.",
                       "LION+CASTLE",
                       "Penguin+Pop+Games+LLC",
                       "FUTURLAB+LIMITED",
                       "Joanne+Hamer",
                       "Tate+Multimedia+Sp.+z+o.o.",
                       "RELEVO+VIDEOGAMES+SL",
                       "WIDE+RIGHT+INTERACTIVE",
                       "ACQUIRE+CORP.",
                       "SOMEQUEST",
                       "FABRICE+BRETON",
                       "NeocoreGames+Distribution+s.r.o.",
                       "TRIPWIRE+INTERACTIVE",
                       "Tetyana+Vysochanska",
                       "GREEN+LAVA+STUDIOS",
                       "NiuGamer+Co.%2C+Ltd.",
                       "EUROVIDEO+MEDIEN+GMBH",
                       "CLICKTEAM+LLC+USA",
                       "STEEL+WOOL+GAMES%2C+LLC",
                       "RIPSTONE+LTD",
                       "4+GLADIATORS",
                       "Gray+Mantis",
                       "BlackMill+Games+B.V.",
                       "HAPPY+PLAYER+PTE.+LTD.",
                       "ROGUESIDE",
                       "Riot+Forge",
                       "Atari+Inc.",
                       "DX+Gameworks+LLC",
                       "ARC+SYSTEM+WORKS",
                       "Domynyo+S.r.l.",
                       "MIXEDBAG+SRL",
                       "NO+MORE+ROBOTS",
                       "Kolosov+Anton+Pavlovich+trading+as+NLB+project",
                       "GAMING+DEV+SRL",
                       "Fireshine+Games",
                       "SNK+CORPORATION",
                       "PLAION",
                       "Kirkidexi+LTD",
                       "Jozef+Debecki",
                       "BLITWORKS+SL",
                       "DEZVOLT+APP+%26+IDEI+SRL",
                       "ODDERS+LAB",
                       "Team+17+Digital+LTD",
                       "PM+Studios",
                       "Serenity+Forge",
                       "hap+Inc.",
                       "Different+Tales+S.A.",
                       "Gaming+Factory+S.A.",
                       "HAILSTORM+GAMES+INC",
                       "OASIS+GAMES+LIMITED",
                       "NIGHT+DIVE+STUDIOS",
                       "10Tons+Ltd.",
                       "Plug+In+Digital",
                       "WHALE+ROCK+GAMES",
                       "Somequest+Sp.+z+o.+o.",
                       "ASPYR+MEDIA",
                       "Fulqrum+Publishing+Ltd.",
                       "SCHELL+GAMES",
                       "Vertigo+Games+B.V",
                       #"Square+Enix+LTD", ############
                       "GAMERA+NEST+SL",
                       "TIKIPOD+PTY+LTD",
                       "7+RAVEN+STUDIOS+CO.+LTD.",
                       "THE+ARCADE+CREW",
                       "SEKAI+PROJECT%2C+INC",
                       "Nicalis+Inc.",
                       "YSBRYD+GAMES+WORLDWIDE+LIMITED",
                       "WOBBLY+TOOTH+LIMITED",
                       "DECK13+INTERACTIVE+GMBH",
                       "Vertigo+Gaming",
                       "Aeternum+Game+Studios+S.L",
                       "Firenut+Consulting+S.L.",
                       "PERPETUAL",
                       "Dear+Villagers",
                       "CORECELL+TECHNOLOGY",
                       "Tunermaxx+Media+GmbH",
                       "Prime+Matter",
                       "WARANÍ+STUDIOS",
                       "ADULT+SWIM",
                       "OOO+VALNAT",
                       "Mega+Cat+Studios",
                       "CREATIVE+VR+3D",
                       "Nextgo24+UG",
                       "Maddy+Makes+Games+Inc.",
                       "Hound+Picked+Ltd",
                       "InXile+Entertainment",
                       "GAIJIN+GAMES",
                       "Good+Shepherd+Entertainment",
                       "DOUBLE+FINE+PRODUCTIONS+INC.",
                       "XSEED+GAMES",
                       "Rogue+Games%2C+Inc.",
                       "Rusinov+Aleksandr+Vladimirovich",
                       "TAKE-TWO+INTERACTIVE+SOFTWARE+UK",
                       "nDreams",
                       "101XP+LTD",
                       "Double+Eleven+Limited",
                       "Leoful",
                       "FLYHIGH+WORKS+CO.%2C+LTD.",
                       
                    ]

In [333]:
publisher = bad_publishers_list[-1]

df_pub = get_pub(publisher)        
        
df = pd.read_csv("full_list_v2.csv")
        
df.loc[df[df['title'].isin(df_pub.title)].index, 'status'] = -1
df = df.sort_values(by=['status', 'title'], ascending=[False, True]).reset_index(drop=True)
df.to_csv("full_list_v2.csv", index=False)

page 1  time 0:00:00.864557


In [334]:
df_pub

,title,platform,psd_URL,icon_URL,price
0,Mercenaries Lament: Requiem of the Silver Wolf,PS5,https://psdeals.net/ua-store/game/2665817/merc...,https://store.playstation.com/store/api/chihir...,649.00₴
1,SHINOBI NON GRATA,PS4,https://psdeals.net/ua-store/game/2634798/shin...,https://store.playstation.com/store/api/chihir...,499.00₴
2,SHIRONE THE DRAGON GIRL,PS4,https://psdeals.net/ua-store/game/2508104/shir...,https://store.playstation.com/store/api/chihir...,489.00₴
3,Transiruby,PS4,https://psdeals.net/ua-store/game/2432131/tran...,https://store.playstation.com/store/api/chihir...,489.00₴
4,Fairune Collection,PS4,https://psdeals.net/ua-store/game/2272971/fair...,https://store.playstation.com/store/api/chihir...,299.00₴
5,Phoenotopia: Awakening,PS4,https://psdeals.net/ua-store/game/2223507/phoe...,https://store.playstation.com/store/api/chihir...,649.00₴


# Sort

In [8]:
df = pd.read_csv("full_list_v2.csv")
df = df.sort_values(by=['status', 'title'], ascending=[False, True]).reset_index(drop=True)
df.to_csv("full_list_v2.csv", index=False)

In [7]:
df.head(10)

,title,platform,psd_URL,icon_URL,price,status
0,A Way Out,PS4,https://psdeals.net/ua-store/game/1480053/a-wa...,https://store.playstation.com/store/api/chihir...,"1,049.00₴",1
1,ASTRO BOT,PS5,https://psdeals.net/ua-store/game/2787643/astr...,https://store.playstation.com/store/api/chihir...,"1,999.00₴",1
2,Alan Wake Remastered,PS5 / PS4,https://psdeals.net/ua-store/game/2263208/alan...,https://store.playstation.com/store/api/chihir...,799.00₴,1
3,Alone in the Dark,PS5,https://psdeals.net/ua-store/game/2599714/alon...,https://store.playstation.com/store/api/chihir...,"1,799.00₴",1
4,Bendy and the Dark Revival,PS4,https://psdeals.net/ua-store/game/2554957/bend...,https://store.playstation.com/store/api/chihir...,929.00₴,1
5,Bendy and the Ink Machine,PS4,https://psdeals.net/ua-store/game/1653327/bend...,https://store.playstation.com/store/api/chihir...,649.00₴,1
6,Burnout™ Paradise Remastered,PS4,https://psdeals.net/ua-store/game/1474936/burn...,https://store.playstation.com/store/api/chihir...,649.00₴,1
7,Ctrl Alt Ego,PS5,https://psdeals.net/ua-store/game/2843851/ctrl...,https://store.playstation.com/store/api/chihir...,819.00₴,1
8,Cuphead and The Delicious Last Course,PS4,https://psdeals.net/ua-store/game/2404934/cuph...,https://store.playstation.com/store/api/chihir...,879.00₴,1
9,Cyberpunk 2077: Полное издание (PS5),PS5 / PS4,https://psdeals.net/ua-store/game/2607614/cybe...,https://store.playstation.com/store/api/chihir...,"2,299.00₴",1


# Delete duplicates

In [4]:
df = pd.read_csv("full_list_v2.csv")
df.drop_duplicates(subset=['title', 'platform', 'psd_URL'], keep=False, inplace=True)
df.to_csv("full_list_v2.csv", index=False)

In [5]:
df.head()

,title,platform,psd_URL,icon_URL,price,status
0,Techno Banter: Ultimate Mix,PS5,https://psdeals.net/ua-store/game/2873228/tech...,https://store.playstation.com/store/api/chihir...,649.00₴,0.0
1,"Warhammer 40,000: Darktide",PS5,https://psdeals.net/ua-store/game/2873227/warh...,https://store.playstation.com/store/api/chihir...,759.00₴,0.0
2,"Warhammer 40,000: Darktide - Imperial Edition",PS5,https://psdeals.net/ua-store/game/2873226/warh...,https://store.playstation.com/store/api/chihir...,"1,139.00₴",0.0
3,Techtonica,PS5,https://psdeals.net/ua-store/game/2873225/tech...,https://store.playstation.com/store/api/chihir...,999.00₴,0.0
4,Skydivers Paradise,PS4,https://psdeals.net/ua-store/game/2873224/skyd...,https://store.playstation.com/store/api/chihir...,64.00₴,0.0


# Liked games

In [ ]:
liked_lst = ['Alone in the Dark',
             "Wreckfest PlayStation®5 Version",
             "Cloudpunk",
             "The Exit 8",
             "Terminator: Resistance",
             "Deliver Us The Moon",
             "RoboCop: Rogue City",
             "Infliction: Extended Cut",
            
            ]